# Importing Libraries

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, GRU, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
from nltk.stem.porter import PorterStemmer
import re
import numpy as np
import tensorflow as tf

In [ ]:
# read csv
data = pd.read_csv('/content/drive/MyDrive/Fake_news/news.csv').iloc[:,1:]
data.head()

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


# Preprocessing Data

In [ ]:
# Merging title and text
data["final_text"] = data["title"] + ' ' + data["text"]
text_data = data['final_text']
labels = data['label']

In [ ]:
# converting labes to 0 and 1
le = LabelEncoder()
labels = le.fit_transform(labels)

In [ ]:
labels

array([0, 0, 1, ..., 0, 1, 1])

In [ ]:
text_data

0       You Can Smell Hillary’s Fear Daniel Greenfield...
1       Watch The Exact Moment Paul Ryan Committed Pol...
2       Kerry to go to Paris in gesture of sympathy U....
3       Bernie supporters on Twitter erupt in anger ag...
4       The Battle of New York: Why This Primary Matte...
                              ...                        
6330    State Department says it can't find emails fro...
6331    The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332    Anti-Trump Protesters Are Tools of the Oligarc...
6333    In Ethiopia, Obama seeks progress on peace, se...
6334    Jeb Bush Is Suddenly Attacking Trump. Here's W...
Name: final_text, Length: 6335, dtype: object

## Downloading Stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# stopwords removal and stemming
ps = PorterStemmer()
stop = stopwords.words('english')

for i in range(0, len(text_data)):
    text_data[i] = re.sub('[^a-zA-Z]', ' ', text_data[i])
    text_data[i] = text_data[i].lower()
    text_data[i] = text_data[i].split()
    text_data[i] = [ps.stem(word) for word in text_data[i] if not word in stopwords.words('english')]
    text_data[i] = ' '.join(text_data[i])

text_data[0]

'smell hillari fear daniel greenfield shillman journal fellow freedom center new york writer focus radic islam final stretch elect hillari rodham clinton gone war fbi word unpreced thrown around often elect ought retir still unpreced nomine major polit parti go war fbi exactli hillari peopl done coma patient wake watch hour cnn hospit bed would assum fbi director jame comey hillari oppon elect fbi attack everyon obama cnn hillari peopl circul letter attack comey current media hit piec lambast target trump surpris clinton alli start run attack ad fbi fbi leadership warn entir left wing establish form lynch mob continu go hillari fbi credibl attack media democrat preemptiv head result investig clinton foundat hillari clinton covert struggl fbi agent obama doj peopl gone explos public new york time compar comey j edgar hoover bizarr headlin jame comey role recal hoover fbi fairli practic admit front spout nonsens boston globe publish column call comey resign outdon time editori claim scan

In [ ]:
max_words = 10000  # Maximum number of words in vocab
max_sequence_length = 200  # Maximum sequence length for padding

In [ ]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(text_data)
sequences = tokenizer.texts_to_sequences(text_data)
X = pad_sequences(sequences, maxlen=max_sequence_length)

In [ ]:
X

array([[1558, 9056,  234, ...,   11,  377,  754],
       [ 142,    8,  202, ...,  796, 1130,  484],
       [1040,  275, 1531, ...,  997,  481, 4878],
       ...,
       [ 267, 2374, 1578, ...,  396,   51,  227],
       [2204,  590,  362, ...,  404, 1444,  944],
       [ 156,  737,  714, ...,    1, 2223, 3549]], dtype=int32)

In [ ]:
#Uncomment to save the array X for next execution
# np.save('/content/drive/MyDrive/Fake_news/X',X)

In [ ]:
#Uncomment the below line to Load the array X from saved state
# X = np.load('/content/drive/MyDrive/Fake_news/X.npy')
X

array([[1558, 9056,  234, ...,   11,  377,  754],
       [ 142,    8,  202, ...,  796, 1130,  484],
       [1040,  275, 1531, ...,  997,  481, 4878],
       ...,
       [ 267, 2374, 1578, ...,  396,   51,  227],
       [2204,  590,  362, ...,  404, 1444,  944],
       [ 156,  737,  714, ...,    1, 2223, 3549]], dtype=int32)

# Train-Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Neural Network Architectures

In [ ]:
embedding_dim = 100
lstm_units = 128

In [ ]:
def LSTM_model():
  model = Sequential()
  model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length))
  model.add(LSTM(units=lstm_units, dropout=0.2, recurrent_dropout=0.2))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [ ]:
def GRU_model():
  model = Sequential()
  model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length))
  model.add(GRU(units=lstm_units, dropout=0.2, recurrent_dropout=0.2))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [ ]:
def Bidirection_model():
  model = Sequential()
  model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length))
  model.add(Bidirectional(LSTM(units=lstm_units, dropout=0.2, recurrent_dropout=0.2)))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

# Training

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

## LSTM

In [ ]:
model = LSTM_model()
history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
                    epochs=20, batch_size=64, callbacks=[early_stopping])

Epoch 1/20
64/64 [==============================] - 94s 1s/step - loss: 0.5166 - accuracy: 0.7479 - val_loss: 0.3486 - val_accuracy: 0.8748
Epoch 2/20
64/64 [==============================] - 82s 1s/step - loss: 0.2028 - accuracy: 0.9265 - val_loss: 0.2888 - val_accuracy: 0.8994
Epoch 3/20
64/64 [==============================] - 77s 1s/step - loss: 0.1347 - accuracy: 0.9613 - val_loss: 0.3458 - val_accuracy: 0.8925
Epoch 4/20
64/64 [==============================] - 83s 1s/step - loss: 0.0567 - accuracy: 0.9832 - val_loss: 0.3681 - val_accuracy: 0.8935
Epoch 5/20
64/64 [==============================] - 86s 1s/step - loss: 0.0193 - accuracy: 0.9948 - val_loss: 0.4231 - val_accuracy: 0.8866


In [ ]:
#Saving the model
model.save('/content/drive/MyDrive/Fake_news/lstm.keras')

In [ ]:
# Uncomment the below line to skip training and load the saved model
# model = tf.keras.models.load_model('/content/drive/MyDrive/Fake_news/lstm.keras')

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

40/40 [==============================] - 3s 69ms/step - loss: 0.2298 - accuracy: 0.9140
Test Loss: 0.2298, Test Accuracy: 0.9140


In [ ]:
new_articles = ["The federal government is rolling out a trusted employer program that is meant to reduce red tape and make it easier for Canadian employers to bring in temporary foreign workers.Officials say the Recognized Employer Pilot program will be open for applications as soon as September, first to employers in agriculture, then to all others starting in January.",
                "Breaking: Aliens have landed!"]
new_sequences = tokenizer.texts_to_sequences(new_articles)
new_padded_sequences = pad_sequences(new_sequences, maxlen=max_sequence_length)
predictions = model.predict(new_padded_sequences)

1/1 [==============================] - 1s 533ms/step


In [ ]:
decoded_predictions = le.inverse_transform(predictions.round().astype(int))
print("Predictions:", decoded_predictions)

Predictions: ['REAL' 'FAKE']


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GRU

In [ ]:
model = GRU_model()
history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
                    epochs=20, batch_size=64, callbacks=[early_stopping])

Epoch 1/20
64/64 [==============================] - 90s 1s/step - loss: 0.5211 - accuracy: 0.7380 - val_loss: 0.4061 - val_accuracy: 0.8156
Epoch 2/20
64/64 [==============================] - 66s 1s/step - loss: 0.1997 - accuracy: 0.9267 - val_loss: 0.3239 - val_accuracy: 0.8846
Epoch 3/20
64/64 [==============================] - 67s 1s/step - loss: 0.0705 - accuracy: 0.9776 - val_loss: 0.4117 - val_accuracy: 0.8787
Epoch 4/20
64/64 [==============================] - 65s 1s/step - loss: 0.0203 - accuracy: 0.9938 - val_loss: 0.4879 - val_accuracy: 0.8649
Epoch 5/20
64/64 [==============================] - 70s 1s/step - loss: 0.0167 - accuracy: 0.9965 - val_loss: 0.6226 - val_accuracy: 0.8757


In [ ]:
model.save('/content/drive/MyDrive/Fake_news/gru.keras')

In [ ]:
# Uncomment the below line to skip training and load the saved model
# model = tf.keras.models.load_model('/content/drive/MyDrive/Fake_news/gru.keras')

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

40/40 [==============================] - 2s 55ms/step - loss: 0.2865 - accuracy: 0.8856
Test Loss: 0.2865, Test Accuracy: 0.8856


In [ ]:
new_articles = ["The federal government is rolling out a trusted employer program that is meant to reduce red tape and make it easier for Canadian employers to bring in temporary foreign workers.Officials say the Recognized Employer Pilot program will be open for applications as soon as September, first to employers in agriculture, then to all others starting in January.",
                "Breaking: Aliens have landed!"]
new_sequences = tokenizer.texts_to_sequences(new_articles)
new_padded_sequences = pad_sequences(new_sequences, maxlen=max_sequence_length)
predictions = model.predict(new_padded_sequences)

1/1 [==============================] - 0s 326ms/step


In [ ]:
decoded_predictions = le.inverse_transform(predictions.round().astype(int))
print("Predictions:", decoded_predictions)

Predictions: ['REAL' 'FAKE']


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Bidirectional

In [ ]:
model = Bidirection_model()
history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
                    epochs=20, batch_size=64, callbacks=[early_stopping])

Epoch 1/20
64/64 [==============================] - 134s 2s/step - loss: 0.5602 - accuracy: 0.7329 - val_loss: 0.3729 - val_accuracy: 0.8432
Epoch 2/20
64/64 [==============================] - 120s 2s/step - loss: 0.2266 - accuracy: 0.9102 - val_loss: 0.2816 - val_accuracy: 0.8935
Epoch 3/20
64/64 [==============================] - 119s 2s/step - loss: 0.0956 - accuracy: 0.9669 - val_loss: 0.2937 - val_accuracy: 0.8895
Epoch 4/20
64/64 [==============================] - 120s 2s/step - loss: 0.0383 - accuracy: 0.9909 - val_loss: 0.4350 - val_accuracy: 0.8748
Epoch 5/20
64/64 [==============================] - 118s 2s/step - loss: 0.0147 - accuracy: 0.9968 - val_loss: 0.4221 - val_accuracy: 0.8886


In [ ]:
model.save('/content/drive/MyDrive/Fake_news/bidir.keras')

In [ ]:
# Uncomment the below line to skip training and load the saved model
#model = tf.keras.models.load_model('/content/drive/MyDrive/Fake_news/bidir.keras')

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

40/40 [==============================] - 4s 108ms/step - loss: 0.2376 - accuracy: 0.9116
Test Loss: 0.2376, Test Accuracy: 0.9116


In [ ]:
new_articles = ["The federal government is rolling out a trusted employer program that is meant to reduce red tape and make it easier for Canadian employers to bring in temporary foreign workers.Officials say the Recognized Employer Pilot program will be open for applications as soon as September, first to employers in agriculture, then to all others starting in January.",
                "Breaking: Aliens have landed!"]
new_sequences = tokenizer.texts_to_sequences(new_articles)
new_padded_sequences = pad_sequences(new_sequences, maxlen=max_sequence_length)
predictions = model.predict(new_padded_sequences)

1/1 [==============================] - 0s 126ms/step


In [ ]:
decoded_predictions = le.inverse_transform(predictions.round().astype(int))
print("Predictions:", decoded_predictions)

Predictions: ['REAL' 'FAKE']


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
